In [32]:
pip install xlsxwriter


Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
import numpy as np
import re


def cleaned_df(csv_file_path, output_format, output_folder):
    df = pd.read_csv(csv_file_path)
    try:
        # Keep only numeric characters in the ProductID column
        df['ProductID'] = df['ProductID'].apply(lambda x: re.sub(r'\D', '', str(x)))

        # replace rows with nan where 'ProductName' is 'INVALID'
        df['ProductName'].replace('INVALID', np.nan, inplace=True)
        df['ProductName'].replace('', np.nan, inplace=True)

        # Replace empty strings with NaN in 'ProductID' column
        df['ProductID'].replace('INVALID', np.nan, inplace=True)
        df['ProductID'].replace('', np.nan, inplace=True)

        df['Category'].replace('INVALID', np.nan, inplace=True)
        df['Category'].replace('', np.nan, inplace=True)

        # replace rows with nan where 'Discount' is 'INVALID'
        df['Discount'].replace('INVALID', np.nan, inplace=True)
        df['Discount'].replace('', np.nan, inplace=True)

        # Fill NaN values in 'ProductName' and 'Category' columns based on mode
        df['ProductName'] = df.groupby('ProductID')['ProductName'].transform(lambda x: x.fillna(x.mode().iloc[0]))
        df['Category'] = df.groupby('ProductName')['Category'].transform(lambda x: x.fillna(x.mode().iloc[0]))

        # Fill NaN values in 'ProductID' columns based on mode
        df['ProductID'] = df.groupby('ProductName')['ProductID'].transform(lambda x: x.fillna(x.mode().iloc[0]))

        # Identify and drop rows with zero values in Price, Quantity, and TotalPrice
        condition_price = (df['Price'] == 0) | (df['TotalPrice'] == 0)
        condition_quantity = (df['Quantity'] == 0) | (df['TotalPrice'] == 0)

        dropped_price_quantity = df[(condition_price) & (condition_quantity)].copy()
        df = df[~(condition_price) & ~(condition_quantity)]
        
        # Output file paths
        cleaned_output_file_path = f"{output_folder}/cleaned_data.{output_format}"
        dropped_output_file_path = f"{output_folder}/dropped_rows.{output_format}"
        
        if output_format.lower() == 'excel':
            try:
                df.to_excel(cleaned_output_file_path, index=False)
                dropped_price_quantity.to_excel(dropped_output_file_path, index=False)
                print(f"Cleaned data saved to: {cleaned_output_file_path}")
                print(f"Dropped rows saved to: {dropped_output_file_path}")
            except Exception as e:
                print("An error occurred during Excel file writing:", str(e))
        elif output_format.lower() == 'csv':
            df.to_csv(cleaned_output_file_path, index=False)
            dropped_price_quantity.to_csv(dropped_output_file_path, index=False)
            print(f"Cleaned data saved to: {cleaned_output_file_path}")
            print(f"Dropped rows saved to: {dropped_output_file_path}")
        else:
            raise ValueError("Output format must be 'excel' or 'csv'")
        
        return df, dropped_price_quantity

    except Exception as e:
        print("An error occurred during data cleaning:", str(e))
        return None,None


In [36]:
cleaned_data,dropped_rows=cleaned_df(r"C:/Users/Admin/Downloads/Shopee_retaiset_conf.csv","csv","C:/Users/Admin/Downloads/clean_shoppe")

Cleaned data saved to: C:/Users/Admin/Downloads/clean_shoppe/cleaned_data.csv
Dropped rows saved to: C:/Users/Admin/Downloads/clean_shoppe/dropped_rows.csv


In [28]:
cleaned_data

,ProductID,ProductName,Price,Category,Discount,Quantity,TotalPrice
1,1,Headphones,126.36,Electronics,0.07,3,379.08
2,1,Headphones,207.21,Electronics,NaN,87,18027.27
3,0,Television,1045.60,Electronics,0.03,49,51234.40
4,2,Phone,520.46,Electronics,0.45,28,14572.88
5,1,Headphones,82.30,Electronics,0.02,48,3950.40
...,...,...,...,...,...,...,...
994,4,T-Shirt,24.54,Clothing,0.15,28,687.12
996,0,Television,1135.04,Electronics,0.02,81,91938.24
997,6,Laptop,1314.22,Electronics,0.25,28,36798.16
998,2,Phone,271.29,Electronics,0.14,17,4611.93


In [26]:
dropped_rows

,ProductID,ProductName,Price,Category,Discount,Quantity,TotalPrice
0,0,Television,0.00,Electronics,0.19,62,0.0
7,3,Toothpaste,0.00,Groceries,NaN,47,0.0
9,4,T-Shirt,0.00,Clothing,0.35,86,0.0
13,0,Television,896.34,Electronics,0.29,0,0.0
16,0,Television,0.00,Electronics,NaN,79,0.0
...,...,...,...,...,...,...,...
985,4,T-Shirt,0.00,Clothing,0.46,16,0.0
986,6,Laptop,587.23,Electronics,0.32,0,0.0
989,0,Television,0.00,Electronics,0.03,3,0.0
993,5,Oven,0.00,HomeDecor,0.29,53,0.0


# Description
1.Data Cleaning Function (clean_data):

Removes non-numeric characters from the 'ProductID' column. Replaces 'INVALID' values with NaN in the 'ProductName' and 'Discount' columns. Fills missing values in 'ProductName' and 'Category' columns based on the mode within each group. Replaces empty strings with NaN in the 'ProductID' column. Fills missing values in the 'ProductID' column based on the mode within each group.

2.Zero Value Dropping Function (drop_zero_values):

Identifies and drops rows with zero values in the 'Price', 'Quantity', or 'TotalPrice' columns.

3.Example Usage:

Reads the dataset from a CSV file. Calls the clean_data function to clean the data. Calls the drop_zero_values function to drop rows with zero values. Prints the cleaned DataFrame without zero values and the dropped rows DataFrame. Prints the original, uncleaned DataFrame for comparison.

